<a href="https://colab.research.google.com/github/ramkumardeepak774/Text_Classification_Flair-vs-Fasttext/blob/main/multi_Output_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive


we will build a multi-output text classification model using the Netflix dataset. The model will classify the input text as either TV Show or Movie. This will be the first output. The model will also classify the rating as: TV-MA, TV-14, TV-PG, R, PG-13 and TV-Y. The rating will be the second output. We will use Scikit-Learn MultiOutputClassifier algorithm to build this model.


In [1]:
import pandas as pd
import numpy as np

In [41]:
from google.colab import files
data = files.upload()

Saving netflix_titles.csv to netflix_titles (1).csv


In [42]:
import io
data = pd.read_csv(io.BytesIO(data["netflix_titles.csv"]))

In [6]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [43]:
df = data[['title', 'description','type', 'rating']]
df.shape

(8807, 4)

In [48]:
df=df.dropna()
df2=df.dropna(axis=0)


In [49]:
df.shape

(8803, 4)

In [50]:
#columns distribution
df['type'].value_counts()


Movie      6129
TV Show    2674
Name: type, dtype: int64

In [51]:
df['rating'].value_counts()


TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
74 min         1
84 min         1
66 min         1
Name: rating, dtype: int64

**Text cleaning** :
For text cleaning, we will convert all our text data into lower case and remove stop words. We will use the NeatText Python package to perform this process. We will install Neattext using the following code:


In [14]:
!pip install neattext


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 3.6 MB/s eta 0:00:00


In [24]:
import neattext.functions as nfx


In [52]:
df['title'] = df['title'].str.lower()


In [53]:
df['title'] = df['title'].apply(nfx.remove_stopwords)


In [54]:
df=df.dropna()


In [55]:
df

,title,description,type,rating
0,dick johnson dead,"As her father nears the end of his life, filmm...",Movie,PG-13
1,blood & water,"After crossing paths at a party, a Cape Town t...",TV Show,TV-MA
2,ganglands,To protect his family from a powerful drug lor...,TV Show,TV-MA
3,jailbirds new orleans,"Feuds, flirtations and toilet talk go down amo...",TV Show,TV-MA
4,kota factory,In a city of coaching centers known to train I...,TV Show,TV-MA
...,...,...,...,...
8802,zodiac,"A political cartoonist, a crime reporter and a...",Movie,R
8803,zombie dumb,"While living alone in a spooky town, a young g...",TV Show,TV-Y7
8804,zombieland,Looking to survive in a world taken over by zo...,Movie,R
8805,zoom,"Dragged from civilian life, a former superhero...",Movie,PG


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier

In [56]:
Xfeatures = df['title']
ylabels = df[['type','rating']]

In [57]:
#To split the dataset into two, use this code:
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=7)

In [34]:
from sklearn.pipeline import Pipeline


In [58]:
#These processes are usually in sequential steps. 
#The output of one process is used as the input of the next process, as shown in the code below.

pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),
                          ('lr_multi',MultiOutputClassifier(LogisticRegression()))])

In [59]:
pipe_lr.fit(x_train,y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('cv', CountVectorizer()),
                ('lr_multi',
                 MultiOutputClassifier(estimator=LogisticRegression()))])

In [60]:
pipe_lr.score(x_test,y_test)


0.2718667171525937